# Promethee III Ranking
This module shows an example of how to use the "Promethee III Ranking" module

## Definition of inputs and problem formalization

In [1]:
import pandas as pd
from core.enums import Direction, GeneralCriterion, FlowType
from modular_parts.preference import compute_preference_indices
from  modular_parts.flows import calculate_promethee_outranking_flows
from modular_parts.ranking import calculate_promethee_iii_ranking

alternatives = [f"a{i}" for i in range(1, 6)]
criteria = [f"g{i}" for i in range(1, 6)]

criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX,
                                 Direction.MAX, Direction.MIN], index=criteria)
generalized_criteria = pd.Series([GeneralCriterion.V_SHAPE, GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE, GeneralCriterion.LEVEL,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE], index=criteria)
criteria_weights = pd.Series([0.35, 0.25, 0.18, 0.07, 0.15], index=criteria)

preference_thresholds = pd.Series([10, 0, 10, 8, 10], index=criteria)
indifference_thresholds = pd.Series([0, 15, 5, 0, 5], index=criteria)

alternatives_performances = pd.DataFrame([[25, 65, 30, 15, 65],
                                          [30, 65, 30, 10, 65],
                                          [50, 30, 60, 55, 50],
                                          [65, 20, 50, 65, 45],
                                          [70, 10, 15, 70, 10]],
                                         index=alternatives, columns=criteria)

alternatives_performances

,g1,g2,g3,g4,g5
a1,25,65,30,15,65
a2,30,65,30,10,65
a3,50,30,60,55,50
a4,65,20,50,65,45
a5,70,10,15,70,10


## Promethee Preference

In [2]:
preferences, partial_preferences = compute_preference_indices(alternatives_performances, preference_thresholds,
                                                              indifference_thresholds,
                                                              pd.Series(data=None, index=criteria, dtype='object'),
                                                              generalized_criteria, criteria_directions,
                                                              criteria_weights)
preferences

,a1,a2,a3,a4,a5
a1,0.000,0.035,0.00,0.00,0.18
a2,0.175,0.000,0.00,0.00,0.18
a3,1.000,1.000,0.00,0.18,0.18
a4,1.000,1.000,0.42,0.00,0.18
a5,0.820,0.820,0.82,0.36,0.00


## Promethee Outranking Flows

In [3]:
outranking_flows = calculate_promethee_outranking_flows(preferences, FlowType.BASIC)
outranking_flows

,positive,negative
a1,0.05375,0.74875
a2,0.08875,0.71375
a3,0.59000,0.31000
a4,0.65000,0.13500
a5,0.70500,0.18000


## Promethee III Ranking

In [4]:
intervals, outranking_pairs = calculate_promethee_iii_ranking(outranking_flows, preferences, 0.5)
intervals

,x,y
a1,-0.916,-0.474
a2,-0.876,-0.374
a3,-0.054,0.614
a4,0.261,0.769
a5,0.378,0.672


In [5]:
outranking_pairs

,a1,a2,a3,a4,a5
a1,I,I,?,?,?
a2,I,I,?,?,?
a3,P,P,I,I,I
a4,P,P,I,I,I
a5,P,P,I,I,I
